In [2]:
# Import packages.
from petsc4py import PETSc
from slepc4py import SLEPc

import numpy as np
import scipy.sparse

from matplotlib import pyplot as plt
import csv
import sys

# Check anaconda environment. Script needs complex petsc/slepc. Scalar type should be complex128.
from petsc4py import PETSc
print("Scalar type: " + str(PETSc.ScalarType))

Scalar type: <class 'numpy.complex128'>


In [3]:
Re = 40
base_dir = './backward_facing_step_compressible/'
case_path = base_dir + str(Re) + '/'

In [4]:
#################################################################################
# Setup Generalized Non Hermitian Eigenvalue Problem
#################################################################################
def setupGeneralizedNonHermitianEPS(LEP, shift):
    LEP.setProblemType(SLEPc.EPS.ProblemType.GNHEP)
    LEP.setType(SLEPc.EPS.Type.KRYLOVSCHUR)
    LEP.setDimensions(10, PETSc.DEFAULT, PETSc.DEFAULT)
    # LEP.setTarget(shift)
    # LEP.setTwoSided(True)
    LEP.setWhichEigenpairs(SLEPc.EPS.Which.LARGEST_REAL)
    # LEP.setWhichEigenpairs(SLEPc.EPS.Which.TARGET_MAGNITUDE)
    LEP.setTolerances(1e-9, 200)

    st = LEP.getST()
    ksp = st.getKSP()

    ksp.setType("preonly")
    pc = ksp.getPC()
    pc.setType("jacobi")

    LEP.setFromOptions()
    LEP.view()

In [5]:
# Load matrices.
L = scipy.sparse.load_npz(case_path + 'L.npz')
A = scipy.sparse.load_npz(case_path + 'A.npz')

# Important: Here equations are formulatet as (\sigma A + L)q = 0.
# However, slepc expects Lq = \sigma A q.
# => Multiply A matrix with -1, before stability analysis.
A = A * (-1)

# Convert from scipy to petsc matrices.
L_pet = PETSc.Mat().createAIJ(size=L.shape, csr=(L.indptr, L.indices, L.data))
A_pet = PETSc.Mat().createAIJ(size=A.shape, csr=(A.indptr, A.indices, A.data))

In [6]:
A_H_pet = A_pet.copy()
A_H_pet.hermitianTranspose()

L_H_pet = L_pet.copy()
L_H_pet.hermitianTranspose()

In [7]:
# Setup eigenvalue problem.
LEP = SLEPc.EPS().create()
LEP.setOperators(L_pet*L_H_pet, A_pet*A_H_pet)
setupGeneralizedNonHermitianEPS(LEP, shift = None)

# Solve eigenvalue problem.
LEP.solve()

# Access solution.
tol, maxit = LEP.getTolerances()
nconv      = LEP.getConverged()
    
print("Stopping condition: tol=%.4g, maxit=%d" % (tol, maxit))
print("Number of converged eigenpairs %d" % nconv)

EPS Object: 1 MPI process
  type: krylovschur
    0% of basis vectors kept after restart
    using the locking variant
  problem type: generalized non-hermitian eigenvalue problem
  selected portion of the spectrum: largest real parts
  postprocessing eigenvectors with purification
  number of eigenvalues (nev): 10
  number of column vectors (ncv): -1
  maximum dimension of projected problem (mpd): -1
  maximum number of iterations: 200
  tolerance: 1e-09
  convergence test: relative to the eigenvalue
BV Object: 1 MPI process
  type: mat
  0 columns of global length -1
  vector orthogonalization method: classical Gram-Schmidt
  orthogonalization refinement: if needed (eta: 0.7071)
  block orthogonalization method: GS
  doing matmult as a single matrix-matrix product
DS Object: 1 MPI process
  type: nhep
ST Object: 1 MPI process
  type: shift
  shift: 0.
  number of matrices: 2
  nonzero pattern of the matrices: UNKNOWN
  KSP Object: (st_) 1 MPI process
    type: preonly
    maximum ite

In [21]:
spectrum = np.zeros(nconv, dtype=complex)


# Create vectors of dimensions of matrix.
evR, evL = L_pet.getVecs()
evL.destroy()
evR_array = []
# evL_array = []

print()
print("        k          ||Ax-kx||/||kx|| ")
print("----------------- ------------------")

for iEv in range(nconv):

    k     = LEP.getEigenvalue(iEv)
    error = LEP.getErrorEstimate(iEv)
        
    LEP.getEigenvector(iEv, evR)
    # LEP.getLeftEigenvector(iEv, evL)

    spectrum[iEv] = k

    if k.imag != 0.0:
        print(" %9f%+9f j %12g" % (k.real, k.imag, error))
    else:
        print(" %12f      %12g" % (k.real, error))

    evR_array.append(evR.copy())
    # evL_array.append(evL.getArray().copy())


        k          ||Ax-kx||/||kx|| 
----------------- ------------------
 213046034360978.281250+0.128360 j  1.75678e-13
 210174855781174.125000-0.331830 j  1.22951e-12
 187566953452064.062500-1064.494465 j  5.15142e-10
 187566227482735.156250+3211.241165 j  4.11074e-10
 187546677468999.250000-26.922485 j  8.67514e-10
 187545952113471.031250+1095.852471 j   5.5919e-10
 187512919521848.031250-754.096207 j  3.61109e-10
 187512195125182.593750+1628.225547 j  7.66004e-10
 187465732566823.968750+94.080281 j  1.20527e-10
 187465009393359.812500-0.145293 j  6.74767e-11


In [22]:
evSVD, _ = L_pet.getVecs()
_.destroy()

evSVD_array = []

for iEv in range(nconv):
    
    A_H_pet.mult(evR_array[iEv], evSVD)
    evSVD_array.append(evSVD.copy())

In [30]:
#check for orthogonality
max_val = 0
for iEv in range(nconv):
    for jEv in range(iEv+1, nconv):
        val = evR_array[iEv].dot(evR_array[jEv])
        print("Orthogonality: ", val)
        max_val = max(max_val, val.real)
print("Max orthogonality: ", max_val)

Orthogonality:  (4.86971378042356e-13+5.387115303525319e-13j)
Orthogonality:  (4.0338338862873285e-11-1.6582789173179567e-11j)
Orthogonality:  (-3.254869627984079e-11+1.1863951206294918e-11j)
Orthogonality:  (5.687372433100956e-11+1.0903263563080977e-10j)
Orthogonality:  (-6.651933127618752e-11+4.2857895381869656e-11j)
Orthogonality:  (-4.123239337255904e-11-4.417779202626923e-11j)
Orthogonality:  (-2.6153812868291032e-11-1.254135409795025e-10j)
Orthogonality:  (1.0153258441998623e-11-8.084385097699615e-12j)
Orthogonality:  (-1.7125069349046267e-12-7.0976070496372256e-12j)
Orthogonality:  (7.521794368365347e-13-2.8297549315012636e-11j)
Orthogonality:  (-2.404440953213163e-13+2.2134057384460322e-11j)
Orthogonality:  (2.953701874766393e-11+8.627053948682598e-13j)
Orthogonality:  (2.0830642848275825e-12+1.8346738247383585e-11j)
Orthogonality:  (-1.1664033880733303e-11+5.271550675465696e-12j)
Orthogonality:  (-2.6940780026806617e-11-2.603171807821054e-12j)
Orthogonality:  (-2.4620837124586

In [31]:
import os

In [33]:
os.makedirs(case_path + 'svd/', exist_ok=True)

In [34]:
for iEv in range(nconv):
    np.savez(case_path + 'svd/' + str(iEv), evR=evR_array[iEv].getArray(), evSVD=evSVD_array[iEv].getArray())

In [ ]:
# for i in range(len(EWs)):
#     sv_data = np.load(case_path + 'svd/' + str(i) + '.npz')
#     svd_np = sv_data['evSVD']

#     equation.q_real_list[equation.dof["u"]].vector().set_local(
#         np.real(svd_np[equation.VMixed.sub(equation.dof["u"]).dofmap().dofs()]))
#     equation.q_imag_list[equation.dof["u"]].vector().set_local(
#         np.imag(svd_np[equation.VMixed.sub(equation.dof["u"]).dofmap().dofs()]))

#     fields_to_write["svd_real"] = equation.q_real_list[equation.dof["u"]]
#     fields_to_write["svd_imag"] = equation.q_imag_list[equation.dof["u"]]

#     # Write eigenmodes and singular vectors.
#     io = Io()
#     io.write_paraview(geometry, settings, "eigen_singular_" + str(i), fields_to_write)

In [9]:
!jupyter nbconvert --to script visualize_eigenmodes.ipynb

[NbConvertApp] Converting notebook visualize_eigenmodes.ipynb to script
[NbConvertApp] Writing 3129 bytes to visualize_eigenmodes.py
